In [2]:
import mysql.connector
import sshtunnel
import pandas as pd
import time

In [3]:
df = pd.read_csv('all_result/reference_full_information_v2.csv',dtype='str')

In [4]:
df_2 = df[['id','year']].reset_index()

In [5]:
df_2.dtypes

index     int64
id       object
year     object
dtype: object

Connect to mysql server

In [ ]:
ssh_host = ...
ssh_port = 22
ssh_username = ...
ssh_password = ...

In [ ]:
mysql_host = '127.0.0.1'
mysql_port = 3306
mysql_username = ...
mysql_password = ...
mysql_database = ...

In [28]:
tunnel = sshtunnel.SSHTunnelForwarder((ssh_host, 22), ssh_password=ssh_password, ssh_username=ssh_username,
     remote_bind_address=('localhost', 3306))
tunnel.start()

In [29]:
conn = mysql.connector.connect(host='127.0.0.1', user=mysql_username, passwd=mysql_password, database=mysql_database, port=tunnel.local_bind_port)
#data = pd.read_sql_query("SHOW DATABASES;", conn)

In [30]:
#mycursor = conn.cursor(buffered=True)
mycursor = conn.cursor()

In [11]:
y = []

for index, row in df_2.iterrows():
    x = tuple(row)
    y.append(x)

y

[(0, 'WOS:000218666500001', '2013'),
 (1, '000444741500122.11', '2017'),
 (2, 'WOS:000331985300047', '2014'),
 (3, 'WOS:000354945400060', '2015'),
 (4, 'WOS:000426811700056', '2017'),
 (5, 'WOS:000326184100037', '2013'),
 (6, '000444741500122.10', '2005'),
 (7, '000444741500122.2', '1000'),
 (8, '000444741500122.4', '2017'),
 (9, '000444741500122.1', '1000'),
 (10, 'WOS:000384659900078', '2016'),
 (11, '000444741500122.6', '2016'),
 (12, 'WOS:000281591300050', '2011'),
 (13, 'WOS:000083294900001', '1999'),
 (14, 'WOS:000335980100002', '2014'),
 (15, 'WOS:000225784800016', '2004'),
 (16, 'WOS:000298360200003', '2011'),
 (17, 'WOS:000292162300025', '2011'),
 (18, 'INSPEC:8850466', '2005'),
 (19, 'WOS:000319047600005', '2013'),
 (20, 'WOS:000249508500060', '2008'),
 (21, '000427524600026.7', '1999'),
 (22, 'WOS:000309602900004', '2012'),
 (23, 'WOS:000203392200004', '1955'),
 (24, 'WOS:000254345900002', '2008'),
 (25, 'WOS:000175082800006', '2002'),
 (26, 'WOS:000293961700004', '2011'),
 

In [13]:
def split(arr, size):
     arrs = []
     while len(arr) > size:
         pice = arr[:size]
         arrs.append(pice)
         arr   = arr[size:]
     arrs.append(arr)
     return arrs

y_1000 = split(y,1000)

upload reference data to mysql server for open_access match

In [14]:
stripe = 'INSERT INTO uwm_reference_data VALUES (%s,%s,%s)'


In [34]:
for i in range(len(y_1000)):
    start = time.time()
    mycursor.executemany(stripe,y_1000[i])
    end = time.time()
    print(i,end-start)

0 0.03024601936340332
1 0.018189430236816406
2 0.06198883056640625
3 0.022998332977294922
4 0.020857810974121094
5 0.06318354606628418
6 0.024396657943725586
7 0.01951432228088379
8 0.059476613998413086
9 0.02299976348876953
10 0.023041963577270508
11 0.06002616882324219
12 0.0222928524017334
13 0.024005889892578125
14 0.0579836368560791
15 0.02678513526916504
16 0.02349996566772461
17 0.020114898681640625
18 0.02352428436279297
19 0.05776834487915039
20 0.020014524459838867
21 0.02481532096862793
22 0.05785489082336426
23 0.02037525177001953
24 0.018358230590820312
25 0.06144285202026367
26 0.01852130889892578
27 0.018830060958862305
28 0.060567617416381836
29 0.02444934844970703
30 0.06784367561340332
31 0.027103662490844727
32 0.020839214324951172
33 0.06302213668823242
34 0.023569583892822266
35 0.02772808074951172
36 0.0662386417388916
37 0.022623300552368164
38 0.020517587661743164
39 0.05699300765991211
40 0.021363019943237305
41 0.023578643798828125
42 0.020980358123779297
43 0

In [36]:
conn.commit()

After get the open_access data of all reference in mysql server, read the results

In [43]:
data_2 = pd.read_sql('SELECT * FROM uwm_reference_OA',conn)

In [49]:
data_2

,id,OST_BK,Is_Open_Access,Type
0,WOS:000283624000009,65801672,1,gold_other
1,WOS:000283624000009,65801672,1,green_published
2,WOS:000283624000009,65801672,1,green_published
3,WOS:000349122100036,83741887,1,gold_doaj
4,WOS:000349122100036,83741887,1,green_published
...,...,...,...,...
640570,WOS:000447243300001,82128409,1,gold_doaj
640571,WOS:000447243300001,82128409,1,green_published
640572,WOS:000447243300001,82128409,1,green_published
640573,WOS:000596506600004,82015787,1,gold_doaj


In [50]:
data_2.drop_duplicates().to_csv('data_mysql/reference_OA.csv',index=False)

2024-10-22 10:51:20,548| ERROR   | Socket exception: An existing connection was forcibly closed by the remote host (10054)
2024-10-22 11:31:39,991| ERROR   | Socket exception: An existing connection was forcibly closed by the remote host (10054)
